In [27]:
#1\. Write a function that converts number representation (bin<->dec<->hex)

def dTOb(a):
    a1=int(a)
    b=""
    c=[]
    if a1>=0:
        b=b+"0"
        b=b+"b"    
    else:
        b=b+"-0"
        b=b+"b"
        a1=a1*-1
    if a1==0:
        b=b+"0"
    else:    
        while a1!=0:
            c=c+[a1%2]
            a1=a1//2
    c.reverse()
    for i in range(len(c)):
        b=b+str(c[i])                   
    print("\nBinary representation is=",b)

def dTOh(a):    
    a1=int(a)
    h=""
    g=[]
    f="abcdef"
    if a1>=0:
        h=h+"0"
        h=h+"x"
    else:
        h=h+"-0"
        h=h+"x"
        a1=a1*-1
    if a1==0:
        h=h+"0"
    else:    
        while a1!=0:
            g=g+[a1%16]
            a1=a1//16
    g.reverse()
    for i in range(len(g)):
        if g[i]>=10:
            h=h+f[g[i]-10]
        else:
            h=h+str(g[i]) 
    print("Hexadecimal representation is=",h)
    
def bTOdh(a):
    a=list(a)
    if a[0]=='-':
        j=1
        a=a[3:]
    else:    
        j=int(a[0])
        a=a[2:]
    c=0
    for i in range(len(a)):
        c=c+(int(a[i]))*2**(len(a)-i-1)
    c=((-1)**j)*c
    print("\nDecimal representation is=",c)
    dTOh(c)
        
def hTObd(a):     
    a=list(a)
    if a[0]=='-':
        j=1
        a=a[3:]
    else:    
        j=int(a[0])
        a=a[2:]
    c=0
    for i in range(len(a)):
        if a[i]=="a":
            c=c+(10)*16**(len(a)-i-1)
        elif a[i]=="b":
            c=c+(11)*16**(len(a)-i-1)
        elif a[i]=="c":
            c=c+(12)*16**(len(a)-i-1)
        elif a[i]=="d":
            c=c+(13)*16**(len(a)-i-1)
        elif a[i]=="e":
            c=c+(14)*(16**(len(a)-i-1))
        elif a[i]=="f":
            c=c+(15)*16**(len(a)-i-1) 
        else:
            c=c+int(a[i])*16**(len(a)-i-1) 
    c=((-1)**j)*c
    print("\nDecimal representation is=",c)
    dTOb(c)
    
def convert(a):
    if (len(a)>2) and (a[1]=="b" or a[2]=="b"):
        print("your representation is Binary. ")
        bTOdh(a)   
    elif (len(a)>2) and (a[1]=="x" or a[2]=="x"):
        print("your representation is Hexadecimal. ")
        hTObd(a)
    else:
        print("your representation is Decimal. ")
        dTOb(a)
        dTOh(a)
        
a={}
a=input("set your representation:")       
convert(a)
     


set your representation:0b00000111
your representation is Binary. 

Decimal representation is= 7
Hexadecimal representation is= 0x7


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
def convert_32bit(s):
    ex=[]
    man=[]
    sin=0
    e=0
    m=1
    for i in range(32):
        if i==0:
            sin=sin+int(s[i])
        elif i<9:    
            ex=ex+[int(s[i])]
        else:    
            man=man+[int(s[i])]
    for i in range(len(ex)):
        e=e+(int(ex[i]))*2**(len(ex)-i-1)    
    for i in range(len(man)):
        m=m+man[22-i]*(2**(-23+i))    
    return ((-1)**sin)*m*(2**(e-127))
print(convert_32bit(input("please enter the word:")))
        
    

please enter the word:00000011111000000000000000000000
1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [29]:
over_f = 1

for i in range(2000):
    over_f = over_f * 2
    print(i,"-->","%10.5e"%(over_f))    
    
    

0 --> 2.00000e+00
1 --> 4.00000e+00
2 --> 8.00000e+00
3 --> 1.60000e+01
4 --> 3.20000e+01
5 --> 6.40000e+01
6 --> 1.28000e+02
7 --> 2.56000e+02
8 --> 5.12000e+02
9 --> 1.02400e+03
10 --> 2.04800e+03
11 --> 4.09600e+03
12 --> 8.19200e+03
13 --> 1.63840e+04
14 --> 3.27680e+04
15 --> 6.55360e+04
16 --> 1.31072e+05
17 --> 2.62144e+05
18 --> 5.24288e+05
19 --> 1.04858e+06
20 --> 2.09715e+06
21 --> 4.19430e+06
22 --> 8.38861e+06
23 --> 1.67772e+07
24 --> 3.35544e+07
25 --> 6.71089e+07
26 --> 1.34218e+08
27 --> 2.68435e+08
28 --> 5.36871e+08
29 --> 1.07374e+09
30 --> 2.14748e+09
31 --> 4.29497e+09
32 --> 8.58993e+09
33 --> 1.71799e+10
34 --> 3.43597e+10
35 --> 6.87195e+10
36 --> 1.37439e+11
37 --> 2.74878e+11
38 --> 5.49756e+11
39 --> 1.09951e+12
40 --> 2.19902e+12
41 --> 4.39805e+12
42 --> 8.79609e+12
43 --> 1.75922e+13
44 --> 3.51844e+13
45 --> 7.03687e+13
46 --> 1.40737e+14
47 --> 2.81475e+14
48 --> 5.62950e+14
49 --> 1.12590e+15
50 --> 2.25180e+15
51 --> 4.50360e+15
52 --> 9.00720e+15
53 

687 --> 1.28421e+207
688 --> 2.56843e+207
689 --> 5.13685e+207
690 --> 1.02737e+208
691 --> 2.05474e+208
692 --> 4.10948e+208
693 --> 8.21896e+208
694 --> 1.64379e+209
695 --> 3.28758e+209
696 --> 6.57517e+209
697 --> 1.31503e+210
698 --> 2.63007e+210
699 --> 5.26014e+210
700 --> 1.05203e+211
701 --> 2.10405e+211
702 --> 4.20811e+211
703 --> 8.41622e+211
704 --> 1.68324e+212
705 --> 3.36649e+212
706 --> 6.73297e+212
707 --> 1.34659e+213
708 --> 2.69319e+213
709 --> 5.38638e+213
710 --> 1.07728e+214
711 --> 2.15455e+214
712 --> 4.30910e+214
713 --> 8.61821e+214
714 --> 1.72364e+215
715 --> 3.44728e+215
716 --> 6.89457e+215
717 --> 1.37891e+216
718 --> 2.75783e+216
719 --> 5.51565e+216
720 --> 1.10313e+217
721 --> 2.20626e+217
722 --> 4.41252e+217
723 --> 8.82504e+217
724 --> 1.76501e+218
725 --> 3.53002e+218
726 --> 7.06003e+218
727 --> 1.41201e+219
728 --> 2.82401e+219
729 --> 5.64803e+219
730 --> 1.12961e+220
731 --> 2.25921e+220
732 --> 4.51842e+220
733 --> 9.03684e+220
734 --> 1.807

OverflowError: int too large to convert to float

In [46]:
under_f = 1
for i in range(2000):
    under_f = under_f / 2
    if under_f==0:
        print(i)
        break

1074


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [45]:

b=1
for i in range(2000):
    b=b/2
    a=1+b
    if a==1.0:
        print(i)
        break

52


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [19]:
def s_q_e_1(a,b,c):
    return (-b+(((b**2)-(4*a*c))**(0.5)))/(2*a) , (-b-(((b**2)-(4*a*c))**(0.5)))/(2*a)
print(s_q_e_1(0.001,1000,0.001),"\n")

def s_q_e_2(a,b,c):
    return ((-b+(((b**2)-(4*a*c))**(0.5)))*(-b-(((b**2)-(4*a*c))**(0.5))))/((2*a)*(-b-(((b**2)-(4*a*c))**(0.5)))),((-b-(((b**2)-(4*a*c))**(0.5)))*((-b+(((b**2)-(4*a*c))**(0.5)))))/((2*a)*((-b+(((b**2)-(4*a*c))**(0.5)))))
print(s_q_e_2(0.001,1000,0.001),"\n")


def s_q_e_3( a, b, c): 
    
    if (b**2-4*a*c)>0: 
        print("roots are real:",(-b+(((b**2)-(4*a*c))**(0.5)))/(2*a),",",(-b-(((b**2)-(4*a*c))**(0.5)))/(2*a),"\n")  
      
    elif (b**2-4*a*c)==0: 
        print("there is one real root:",-b / (2 * a),"\n") 
         
    else:
        print("roots are Complex:",-b/(2*a), " + i",((b**2)-(4*a*c))**(0.5),",",-b/(2*a), " - i",((b**2)-(4*a*c))**(0.5)) 
s_q_e_3(0.001,1000,0.001)


(-9.999894245993346e-07, -999999.999999) 

(-9.999894245993346e-07, -999999.9999990001) 

roots are real: -9.999894245993346e-07 , -999999.999999 



6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [51]:
def f_x(x):
    return x*(x-1)
def d_f_x(x):
    return (2*x)-1
def l_f_x(x,delta):
    return (f_x(x+delta)-f_x(x))/delta
print(d_f_x(1))
print(l_f_x(1,10**(-2)),"\n")
print(10**(-4),"-->",l_f_x(1,10**(-4))-1)
print(10**(-6),"-->",l_f_x(1,10**(-6))-1)
print(10**(-8),"-->",l_f_x(1,10**(-8))-1)
print(10**(-10),"-->",l_f_x(1,10**(-10))-1)
print(10**(-12),"-->",l_f_x(1,10**(-12))-1)
print(10**(-14),"-->",l_f_x(1,10**(-14))-1)


#a: becuase the 𝛿=10^−2 is not enough small and it is not approxtimatly equal to zero.

#b: untill 10^−8 our accuracy increases but after that the accuracy deacrases.


1
1.010000000000001 

0.0001 --> 9.999999988985486e-05
1e-06 --> 9.99917733279787e-07
1e-08 --> 3.922528746258536e-09
1e-10 --> 8.284037100736441e-08
1e-12 --> 8.890058334132256e-05
1e-14 --> -0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [42]:
n=100
h=2/n
i=0
for j in range(1,n+1):
    i=i+h*((1-(-1+(j*h))**2)**(0.5))
print("difrence between this result and I::",i-1.57079632679) 
print(i,"\n")
%timeit i
# it works good and the result is near to the exact result    

# 20.7 ns ± 0.269 ns per loop so if we want to run less than 1 second , N must be less than 5*10**(7)
# and for run less than 1 minute , N must be less than 3*10**(9)

difrence between this result and I:: -0.0016620712407495741
1.5691342555492505 

20.8 ns ± 0.469 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
